# Import

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import json
#import seaborn as sns
import os

import random
from tqdm.notebook import tqdm
#import plotly.express as px

#import lightgbm as lgb
import copy
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader, random_split
from torch.nn import functional as F
from torch.optim import lr_scheduler

import RNA

os.getcwd()

'/Users/ishanshah/Documents/covid-mrna-degradation'

# Settings

In [57]:
class config:
    train_file = '/home/veer/covid-mrna-degradation/data/train.json'
    test_file = '/home/veer/covid-mrna-degradation/data/test.json'
    pretrain_dir = './'
    sample_submission = '/home/veer/covid-mrna-degradation/data/sample_submission.csv'
    learning_rate = 0.001
    batch_size = 64
    n_epoch = 20
    n_split = 2
    K = 1 # number of aggregation loop (also means number of GCN layers)
    gcn_agg = 'mean' # aggregator function: mean, conv, lstm, pooling
    filter_noise = True
    seed = 1234

In [58]:
class config:
    train_file = '/Users/ishanshah/Documents/covid-mrna-degradation/data/train.json'
    test_file = '/Users/ishanshah/Documents/covid-mrna-degradation/data/test.json'
    pretrain_dir = './'
    sample_submission = '//Users/ishanshah/Documents/covid-mrna-degradation/data/sample_submission.csv'
    learning_rate = 0.001
    batch_size = 64
    n_epoch = 20
    n_split = 2
    K = 1 # number of aggregation loop (also means number of GCN layers)
    gcn_agg = 'mean' # aggregator function: mean, conv, lstm, pooling
    filter_noise = True
    seed = 1234

# Utils

In [59]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [60]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(config.seed)

# Define model

In [72]:
class GCN(nn.Module):
    '''
    Implementation of one layer of GraphSAGE
    '''
    def __init__(self, input_dim, output_dim, aggregator=config.gcn_agg):
        super(GCN, self).__init__()
        self.aggregator = aggregator
        
        if aggregator == 'mean':
            linear_input_dim = input_dim * 2
        elif aggregator == 'conv':
            linear_input_dim = input_dim
#         elif aggregator == 'pooling':
#             linear_input_dim = input_dim * 2
#             self.linear_pooling = nn.Linear(input_dim, input_dim)
        elif aggregator == 'lstm':
            self.lstm_hidden = 64
            linear_input_dim = input_dim + self.lstm_hidden
            self.lstm_agg = nn.LSTM(input_dim, self.lstm_hidden, num_layers=1, batch_first=True)
        
        self.linear_gcn = nn.Linear(in_features=linear_input_dim, out_features=output_dim)
        
    def forward(self, input_, adj_matrix):
        if self.aggregator == 'conv':
            # set elements in diagonal of adj matrix to 1 with conv aggregator
            idx = torch.arange(0, adj_matrix.shape[-1], out=torch.LongTensor())
            adj_matrix[:, idx, idx] = 1
            
        adj_matrix = adj_matrix.type(torch.float32)
        sum_adj = torch.sum(adj_matrix, axis=2)
        sum_adj[sum_adj==0] = 1
        
        if self.aggregator == 'mean' or self.aggregator == 'conv':
            feature_agg = torch.bmm(adj_matrix, input_)
            feature_agg = feature_agg / sum_adj.unsqueeze(dim=2)
            
#         elif self.aggregator == 'pooling':
#             feature_pooling = self.linear_pooling(input_)
#             feature_agg = torch.sigmoid(feature_pooling)
#             feature_agg = torch.bmm(adj_matrix, feature_agg)
#             feature_agg = feature_agg / sum_adj.unsqueeze(dim=2)

        elif self.aggregator == 'lstm':
            feature_agg = torch.zeros(input_.shape[0], input_.shape[1], self.lstm_hidden).cuda()
            for i in range(adj_matrix.shape[1]):
                neighbors = adj_matrix[:, i, :].unsqueeze(2) * input_
                _, hn = self.lstm_agg(neighbors)
                feature_agg[:, i, :] = torch.squeeze(hn[0], 0)
                
        if self.aggregator != 'conv':
            feature_cat = torch.cat((input_, feature_agg), axis=2)
        else:
            feature_cat = feature_agg
                
        feature = torch.sigmoid(self.linear_gcn(feature_cat))
        feature = feature / torch.norm(feature, p=2, dim=2).unsqueeze(dim=2)
        
        return feature
    
class Net(nn.Module):
    def __init__(self, num_embedding=14, seq_len=107, pred_len=68, dropout=0.5, 
                 embed_dim=100, hidden_dim=128, K=1, aggregator='mean'):
        '''
        K: number of GCN layers
        aggregator: type of aggregator function
        '''
        print("using aggregate function %s"%aggregator)
        super(Net, self).__init__()
        
        self.pred_len = pred_len
        self.embedding_layer = nn.Embedding(num_embeddings=num_embedding, 
                                      embedding_dim=embed_dim)
        
        self.gcn = nn.ModuleList([GCN(3 * embed_dim, 3 * embed_dim, aggregator=aggregator) for i in range(K)])
        
        self.gru_layer = nn.GRU(input_size=3 * embed_dim, 
                          hidden_size=hidden_dim, 
                          num_layers=3, 
                          batch_first=True, 
                          dropout=dropout, 
                          bidirectional=True)
        
        self.linear_layer = nn.Linear(in_features=2 * hidden_dim, 
                                out_features=5)
        
    def forward(self, input_, adj_matrix):
        #embedding
        embedding = self.embedding_layer(input_)
        embedding = torch.reshape(embedding, (-1, embedding.shape[1], embedding.shape[2] * embedding.shape[3]))
        
        #gcn
        gcn_feature = embedding
        for gcn_layer in self.gcn:
            gcn_feature = gcn_layer(gcn_feature, adj_matrix)
        
        #gru
        gru_output, gru_hidden = self.gru_layer(gcn_feature)
        truncated = gru_output[:, :self.pred_len]
        
        output = self.linear_layer(truncated)
        
        return output

# Load Data

In [73]:
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

In [74]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

def get_couples(structure):
    """
    For each closing parenthesis, I find the matching opening one and store their index in the couples list.
    The assigned list is used to keep track of the assigned opening parenthesis
    """
    opened = [idx for idx, i in enumerate(structure) if i == '(']
    closed = [idx for idx, i in enumerate(structure) if i == ')']

    assert len(opened) == len(closed)
    assigned = []
    couples = []

    for close_idx in closed:
        for open_idx in opened:
            if open_idx < close_idx:
                if open_idx not in assigned:
                    candidate = open_idx
            else:
                break
        assigned.append(candidate)
        couples.append([candidate, close_idx])
        
    assert len(couples) == len(opened)
    
    return couples

def build_matrix(couples, size):
    mat = np.zeros((size, size))
    
    for i in range(size):  # neigbouring bases are linked as well
        if i < size - 1:
            mat[i, i + 1] = 1
        if i > 0:
            mat[i, i - 1] = 1
    
    for i, j in couples:
        mat[i, j] = 1
        mat[j, i] = 1
        
    return mat

def convert_to_adj(structure):
    couples = get_couples(structure)
    mat = build_matrix(couples, len(structure))
    return mat

def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    inputs = np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [token2int[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )
    
    adj_matrix = np.array(df['structure'].apply(convert_to_adj).values.tolist())
    
    return inputs, adj_matrix

In [75]:
train = pd.read_json(config.train_file, lines=True)

if config.filter_noise:
    train = train[train.signal_to_noise > 1]
    
test = pd.read_json(config.test_file, lines=True)
sample_df = pd.read_csv(config.sample_submission)

In [76]:
seqs = list(train['sequence'])
seqs[0]

'GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC'

In [77]:
def get_predicted_loop_type(sequence, structure, debug=False):
    !echo $sequence > a.dbn
    !echo "$structure" >> a.dbn
    !export PERL5LIB=/root/perl5/lib/perl5 && perl bpRNA/bpRNA.pl a.dbn
    result = [l.strip('\n') for l in open('a.st')]
    if debug:
        print(sequence)
        print(structure)
        print(result[5])
    return result

In [95]:
train.shape[0]

2096

In [96]:
import random
import time
random.seed(12345)
sample_inds = random.sample(list(range(2096)), 250)
sample_inds = np.array(sorted(sample_inds))
train_sample = train.reset_index().iloc[sample_inds, :]

In [97]:
sample_inds

array([   8,   13,   14,   27,   38,   41,   47,   52,   56,   62,   69,
         96,  100,  110,  111,  112,  120,  125,  129,  136,  144,  155,
        164,  165,  167,  170,  180,  187,  202,  206,  209,  232,  241,
        242,  249,  252,  270,  277,  278,  309,  318,  321,  327,  335,
        340,  371,  373,  375,  384,  392,  394,  395,  407,  408,  433,
        440,  449,  454,  465,  486,  488,  490,  492,  497,  508,  513,
        531,  553,  557,  585,  600,  606,  608,  613,  614,  615,  620,
        639,  642,  648,  652,  657,  662,  664,  666,  669,  670,  672,
        675,  682,  713,  714,  720,  728,  729,  732,  735,  738,  744,
        745,  754,  757,  761,  765,  769,  777,  778,  793,  798,  822,
        825,  826,  832,  842,  844,  849,  853,  854,  876,  882,  886,
        899,  901,  909,  938,  944,  951,  952,  955,  988,  989,  996,
       1019, 1055, 1061, 1064, 1069, 1082, 1085, 1106, 1116, 1126, 1163,
       1173, 1180, 1197, 1222, 1223, 1229, 1232, 12

In [79]:
int2token = {v:k for k,v in token2int.items()}

In [80]:
# DO ALL THIS BEFORE THE PERTURBATIONS:
base_pairs = {3:'A', 4:'C', 5:'G', 6:'U'}
train_inputs, train_adj = preprocess_inputs(train_sample)
train_labels = np.array(train[pred_cols].values.tolist()).transpose((0, 2, 1))

num_seqs = train_inputs.shape[0] 
seq_len = train_inputs.shape[1]

train_adj = torch.tensor(train_adj, dtype=torch.float32, requires_grad=False)
train_labels = torch.tensor(train_labels, dtype=torch.float32)

#embedding
embedding_layer = nn.Embedding(num_embeddings=14, 
                                      embedding_dim=100)

results_dict = {}

every_5_bases = [i for i in range(seq_len) if i%5 == 0]

In [83]:
model_path = "/Users/ishanshah/Documents/covid-mrna-degradation/GCN_GRU/GCN_mean_patience10/gcn_gru_4.pt"
model = Net(seq_len=107, pred_len=107, K=config.K, aggregator=config.gcn_agg)
model.load_state_dict(torch.load(model_path, map_location='cpu'))

# EXECUTE THE PERTURBATIONS
for i in every_5_bases:
    orig_bp = copy.deepcopy(train_inputs[:,i,0])
    orig_struct = copy.deepcopy(train_inputs[:,:,1])
    orig_loop = copy.deepcopy(train_inputs[:,:,2])    
    results_dict[i] = {}
    for base in [3,4,5,6]:
        print("Base = {}; Sequence Position = {}".format(int2token[base], i))
        train_inputs[:,i,0] = base        
        pert_seqs = [''.join(i) for i in np.vectorize(int2token.get)(train_inputs[:,:,0])]
        
        #t = time.time()
        
        pert_secondaries = []
        pert_loop_types = []
        
        iters = 0
        for s in pert_seqs:
            secondary = RNA.fold(s)[0]
            secondary_ints = [token2int[i] for i in list(secondary)]
            pert_secondaries.append(secondary_ints)
            res = get_predicted_loop_type(s, secondary, debug=False)
            loop_type_ints = [token2int[i] for i in list(res[5])]
            pert_loop_types.append(loop_type_ints)
            iters += 1
            
            if iters % 100 == 0:
                print(str(iters) + " sequences done.")
         
        train_inputs[:,:,1] = np.array(pert_secondaries)
        train_inputs[:,:,2] = np.array(pert_loop_types)
    
        #print("Time:", time.time() - t)
        
        # Convert to embedding and get it into training format
        train_inputs_tens = torch.tensor(train_inputs, dtype=torch.long)
        train_inputs_tens = train_inputs_tens.clone().detach().requires_grad_(False)
        
        print("Forward pass starting:")
        model.eval()
        t = time.time()
        preds = model(train_inputs_tens, train_adj)
        preds = preds.cpu().detach().numpy()
        print("Forward pass time:", time.time() - t)
        results_dict[i][base_pairs[base]] = preds
        # DO FORWARD PASS. (forward pass function)
        # results_dict[i][base_pairs[base]] = results list of 5(?)
    train_inputs[:,i,0] = orig_bp
    train_inputs[:,:,1] = orig_struct
    train_inputs[:,:,2] = orig_loop

    #if i == 10:
        #break

using aggregate function mean
Base = A; Sequence Position = 0
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.4669179916381836
Base = C; Sequence Position = 0
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.608271837234497
Base = G; Sequence Position = 0
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.3104510307312012
Base = U; Sequence Position = 0
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.180577039718628
Base = A; Sequence Position = 5
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.214777946472168
Base = C; Sequence Position = 5
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.1793489456176758
Base = G; Sequence Position = 5
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.197394847869873
Base = U; Sequence Position = 5
100 sequenc

Forward pass time: 1.166515827178955
Base = C; Sequence Position = 75
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.1759600639343262
Base = G; Sequence Position = 75
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.1297581195831299
Base = U; Sequence Position = 75
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.2026910781860352
Base = A; Sequence Position = 80
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.1879870891571045
Base = C; Sequence Position = 80
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.1824660301208496
Base = G; Sequence Position = 80
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.629241943359375
Base = U; Sequence Position = 80
100 sequences done.
200 sequences done.
Forward pass starting:
Forward pass time: 1.8261339664459229
Base = A; Sequence Positio

In [84]:
results_dict

{0: {'A': array([[[ 4.16235149e-01,  4.04615372e-01,  2.25840116e+00,
            3.45499784e-01,  6.18701637e-01],
          [ 1.00560939e+00,  1.34839296e+00,  2.50353956e+00,
            1.20522416e+00,  1.34742177e+00],
          [ 1.01923370e+00,  6.52383268e-01,  9.87840891e-01,
            9.63504970e-01,  1.19522607e+00],
          ...,
          [ 3.22969317e-01,  1.26213253e-01, -3.94907296e-02,
            1.21013202e-01,  2.46659160e-01],
          [ 5.85028380e-02,  3.68151441e-02, -2.86091268e-01,
            6.52546063e-02, -2.38685347e-02],
          [-1.24186665e-01, -1.55611128e-01, -7.05619454e-01,
           -1.82703584e-01, -3.11620831e-01]],
  
         [[ 5.42197466e-01,  6.72395706e-01,  1.81090796e+00,
            6.68633461e-01,  7.28606880e-01],
          [ 1.45411956e+00,  2.72437358e+00,  3.03472686e+00,
            2.60136342e+00,  2.04585624e+00],
          [ 1.34276831e+00,  1.13310611e+00,  1.15244889e+00,
            1.52869928e+00,  1.29798162e+00],
 

In [94]:
from random import sample
import json

In [91]:
f = open("/Users/ishanshah/Documents/covid-mrna-degradation/inference/results_dict_250.txt","w")
f.write( str(results_dict) )
f.close()
# View the effects of a randomly selected sequence
seq = sample(every_5_bases, 1)
output = 0
mean_effect = [0 for i in every_5_bases]
for ind, i in enumerate(every_5_bases):
    theta = 0
    for b in [3, 4, 5, 6]:
        theta = theta + results_dict[i][base_pairs[b]][seq][0][i][output]
    theta = theta / 4.0
    mean_effect[ind] = theta
# get original predictions
train_seq = train_inputs[seq,:,:]
train_seq_tens = torch.tensor(train_seq, dtype=torch.long)
train_seq_tens = train_seq_tens.clone().detach().requires_grad_(False)
model.eval()
preds = model(train_seq_tens, train_adj[seq,:,:])
preds = preds.cpu().detach().numpy()
# compute normalized effect
norm_effect = preds[0,:,output] - mean_effect
# plot effect
plt.plot(range(seq_len), norm_effect)
plt.ylabel('Normalized Effect')
plt.xlabel('Sequence Position')
plt.suptitle('Normalized Effect for Sequence ' + str(seq[0]))
plt.savefig('norm_effect_plot_' + str(seq[0]) + '.png')
plt.show()

ValueError: operands could not be broadcast together with shapes (107,) (22,) 

In [100]:
import pickle

In [101]:
with open('inference/results_dict_250.p', 'wb') as f:
    pickle.dump(results_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [102]:
with open('inference/results_dict_250.p', 'rb') as fp:
    rd250 = pickle.load(fp)

In [36]:
a = np.array([[3,4,6,3], [4,6,6,3]])

In [39]:
[''.join(g) for g in (np.vectorize(base_pairs.get)(a))]

['ACUA', 'CUUA']